In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
import time

# Set random seed for reproducibility
np.random.seed(42)

def load_data(file_path):
    df = pd.read_csv(file_path)
    print(f"Loaded data with shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    return df

def preprocess_text(text):
    if isinstance(text, str):
        # Remove HTML tags
        text = re.sub(r'<.*?>', '', text)
        # Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        # Remove special characters and numbers
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)
        # Convert to lowercase
        text = text.lower()
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ""

def preprocess_data(df):
    df['processed_content'] = df['Comment'].apply(preprocess_text)
    return df

def vectorize_text(texts, max_features=5000):
    vectorizer = TfidfVectorizer(max_features=max_features)
    X = vectorizer.fit_transform(texts)
    return X, vectorizer

def apply_clustering(X, n_clusters=3):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(X)
    return clusters

def map_clusters_to_sentiments(df, clusters):
    cluster_sentiments = {
        0: 'positive',
        1: 'neutral',
        2: 'negative'
    }
    
    df['cluster'] = clusters
    df['sentiment'] = df['cluster'].map(cluster_sentiments)
    sentiment_map = {'positive': 2, 'neutral': 1, 'negative': 0}
    df['sentiment_label'] = df['sentiment'].map(sentiment_map)
    
    return df

def evaluate_boost_models(X_train, X_test, y_train, y_test):
    models = {
        'XGBoost': xgb.XGBClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=6,
            min_child_weight=1,
            gamma=0,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42
        ),
        'AdaBoost': AdaBoostClassifier(
            n_estimators=100,
            learning_rate=0.1,
            random_state=42
        )
    }
    
    results = {}
    
    for name, model in models.items():
        print(f"\nTraining {name}...")
        start_time = time.time()
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        predictions = model.predict(X_test)
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, predictions)
        training_time = time.time() - start_time
        
        results[name] = {
            'accuracy': accuracy,
            'training_time': training_time,
            'predictions': predictions,
            'model': model
        }
        
        # Print results
        print(f"{name} Results:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Training Time: {training_time:.2f} seconds")
        print("\nClassification Report:")
        print(classification_report(y_test, predictions))
        
        # Plot confusion matrix
        plot_confusion_matrix(y_test, predictions, name)
        
        # Plot feature importance
        if hasattr(model, 'feature_importances_'):
            plot_feature_importance(model, vectorizer, name)
    
    return results

def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Negative', 'Neutral', 'Positive'],
                yticklabels=['Negative', 'Neutral', 'Positive'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix - {title}')
    plt.tight_layout()
    plt.savefig(f'confusion_matrix_{title.replace(" ", "_").lower()}.png')
    plt.close()

def plot_feature_importance(model, vectorizer, model_name, top_n=20):
    # Get feature importance scores
    importance = model.feature_importances_
    
    # Get feature names
    feature_names = vectorizer.get_feature_names_out()
    
    # Create dataframe of features and their importance scores
    feature_importance = pd.DataFrame({
        'feature': feature_names,
        'importance': importance
    })
    
    # Sort and get top features
    feature_importance = feature_importance.sort_values('importance', ascending=False).head(top_n)
    
    # Plot
    plt.figure(figsize=(12, 6))
    sns.barplot(data=feature_importance, x='importance', y='feature')
    plt.title(f'Top {top_n} Most Important Features - {model_name}')
    plt.xlabel('Importance Score')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.savefig(f'feature_importance_{model_name.lower()}.png')
    plt.close()

def plot_model_comparison(results):
    # Prepare data for plotting
    models = list(results.keys())
    accuracies = [results[model]['accuracy'] for model in models]
    times = [results[model]['training_time'] for model in models]
    
    # Plot accuracies
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    bars = plt.bar(models, accuracies, color=['blue', 'green'])
    plt.title('Model Accuracy Comparison')
    plt.ylabel('Accuracy')
    for bar, acc in zip(bars, accuracies):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{acc:.4f}', ha='center', va='bottom')
    
    # Plot training times
    plt.subplot(1, 2, 2)
    bars = plt.bar(models, times, color=['blue', 'green'])
    plt.title('Training Time Comparison')
    plt.ylabel('Time (seconds)')
    for bar, time_val in zip(bars, times):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                f'{time_val:.2f}s', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.savefig('model_comparison.png')
    plt.close()

def main(file_path):
    # Load and preprocess data
    df = load_data(file_path)
    df = preprocess_data(df)
    
    # Vectorize the text
    print("\nVectorizing text data...")
    global vectorizer  # Make vectorizer accessible to feature importance plotting
    X, vectorizer = vectorize_text(df['processed_content'])
    
    # Apply clustering and get sentiments
    print("Applying clustering for sentiment analysis...")
    clusters = apply_clustering(X)
    df = map_clusters_to_sentiments(df, clusters)
    
    # Split the data
    print("Splitting data into train and test sets...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, df['sentiment_label'], test_size=0.2, random_state=42, stratify=df['sentiment_label']
    )
    
    # Train and evaluate boost models
    results = evaluate_boost_models(X_train, X_test, y_train, y_test)
    
    # Plot model comparison
    plot_model_comparison(results)
    
    return results, df

if __name__ == "__main__":
    file_path = "cleaned_english_yt_comments.csv"
    results, df = main(file_path)
    
    print("\nFinal Accuracy Summary:")
    for model, result in results.items():
        print(f"{model}: {result['accuracy']:.4f}")

Loaded data with shape: (9490, 3)
Columns: ['Serial Number', 'Published At', 'Comment']

Vectorizing text data...
Applying clustering for sentiment analysis...
Splitting data into train and test sets...

Training XGBoost...
XGBoost Results:
Accuracy: 0.9563
Training Time: 16.88 seconds

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.97      0.97       240
           1       0.96      0.90      0.93       585
           2       0.95      0.98      0.97      1073

    accuracy                           0.96      1898
   macro avg       0.96      0.95      0.96      1898
weighted avg       0.96      0.96      0.96      1898


Training AdaBoost...


D:\Documents\Zoom\anconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Results:
Accuracy: 0.8973
Training Time: 5.66 seconds

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.95      0.94       240
           1       0.79      0.92      0.85       585
           2       0.96      0.87      0.92      1073

    accuracy                           0.90      1898
   macro avg       0.89      0.91      0.90      1898
weighted avg       0.91      0.90      0.90      1898


Final Accuracy Summary:
XGBoost: 0.9563
AdaBoost: 0.8973


In [4]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
import time
from sklearn.cluster import KMeans

# Set random seed for reproducibility
np.random.seed(42)

def load_data(file_path):
    df = pd.read_csv(file_path)
    df.dropna(subset=['Comment'], inplace=True)  # Remove empty rows
    print(f"Loaded data with shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    return df

def preprocess_text(text):
    if isinstance(text, str):
        text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # Remove URLs
        text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
        text = re.sub(r'\d+', '', text)  # Remove numbers
        text = text.lower().strip()  # Convert to lowercase and strip whitespace
        return text
    return ""

def preprocess_data(df):
    df['processed_content'] = df['Comment'].apply(preprocess_text)
    return df

def vectorize_text(texts, max_features=5000):
    vectorizer = TfidfVectorizer(max_features=max_features)
    X = vectorizer.fit_transform(texts)
    return X, vectorizer

def apply_clustering(X, n_clusters=3):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(X)
    return clusters

def map_clusters_to_sentiments(df, clusters):
    cluster_sentiments = {0: 'positive', 1: 'neutral', 2: 'negative'}
    df['cluster'] = clusters
    df['sentiment'] = df['cluster'].map(cluster_sentiments)
    sentiment_map = {'positive': 2, 'neutral': 1, 'negative': 0}
    df['sentiment_label'] = df['sentiment'].map(sentiment_map)
    return df

def evaluate_boost_models(X_train, X_test, y_train, y_test):
    models = {
        'XGBoost': xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=6, random_state=42),
        'AdaBoost': AdaBoostClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
    }
    results = {}
    for name, model in models.items():
        print(f"\nTraining {name}...")
        start_time = time.time()
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        training_time = time.time() - start_time
        results[name] = {'accuracy': accuracy, 'training_time': training_time, 'predictions': predictions, 'model': model}
        print(f"{name} Accuracy: {accuracy:.4f}")
    return results

def save_results(df, filename="YT_processed_results_ANALYSIS.csv"):
    df.to_csv(filename, index=False)
    print(f"Results saved to {filename}")

def main(file_path):
    df = load_data(file_path)
    df = preprocess_data(df)
    X, vectorizer = vectorize_text(df['processed_content'])
    clusters = apply_clustering(X)
    df = map_clusters_to_sentiments(df, clusters)
    X_train, X_test, y_train, y_test = train_test_split(X, df['sentiment_label'], test_size=0.2, random_state=42, stratify=df['sentiment_label'])
    results = evaluate_boost_models(X_train, X_test, y_train, y_test)
    save_results(df)
    return results, df

if __name__ == "__main__":
    file_path = "cleaned_english_yt_comments.csv"
    results, df = main(file_path)
    print("\nFinal Accuracy Summary:")
    for model, result in results.items():
        print(f"{model}: {result['accuracy']:.4f}")


Loaded data with shape: (9475, 3)
Columns: ['Serial Number', 'Published At', 'Comment']

Training XGBoost...
XGBoost Accuracy: 0.9546

Training AdaBoost...


D:\Documents\Zoom\anconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoost Accuracy: 0.8892
Results saved to YT_processed_results_ANALYSIS.csv

Final Accuracy Summary:
XGBoost: 0.9546
AdaBoost: 0.8892


In [6]:
import pandas as pd

# Load the dataset
file_path = "YT_processed_results_ANALYSIS.csv"
df = pd.read_csv(file_path)

# Remove empty rows
df = df.dropna()

# Select numeric columns for statistical analysis
numeric_cols = df.select_dtypes(include=['number'])

# Compute descriptive statistics
stats = {
    "Mean": numeric_cols.mean(),
    "Median": numeric_cols.median(),
    "Mode": numeric_cols.mode().iloc[0],
    "Standard Deviation": numeric_cols.std(),
    "Standard Error": numeric_cols.sem(),
    "Skewness": numeric_cols.skew(),
    "Kurtosis": numeric_cols.kurtosis(),
    "25th Percentile": numeric_cols.quantile(0.25),
    "50th Percentile (Median)": numeric_cols.quantile(0.50),
    "75th Percentile": numeric_cols.quantile(0.75),
}

# Convert to DataFrame for better readability
stats_df = pd.DataFrame(stats)

# Display results
print(stats_df)

# Save results to a CSV file
stats_df.to_csv("descriptive_statistics.csv", index=True)
print("Descriptive statistics saved to descriptive_statistics.csv")


                        Mean  Median  Mode  Standard Deviation  \
Serial Number    4746.750211  4748.5   1.0         2740.073941   
cluster             0.550275     0.0   0.0            0.711839   
sentiment_label     1.449725     2.0   2.0            0.711839   

                 Standard Error  Skewness  Kurtosis  25th Percentile  \
Serial Number         28.160033 -0.001091 -1.199335          2376.75   
cluster                0.007316  0.900781 -0.516392             0.00   
sentiment_label        0.007316 -0.900781 -0.516392             1.00   

                 50th Percentile (Median)  75th Percentile  
Serial Number                      4748.5          7118.25  
cluster                               0.0             1.00  
sentiment_label                       2.0             2.00  
Descriptive statistics saved to descriptive_statistics.csv


In [12]:
import pandas as pd

# Load the dataset
file_path = "YT_processed_results_ANALYSIS.csv"
df = pd.read_csv(file_path)

# Ensure there are no empty rows
df = df.dropna(subset=['sentiment_label'])

# Compute descriptive statistics for the sentiment_label column
descriptive_stats = {
    "Mean": df['sentiment_label'].mean(),
    "Median": df['sentiment_label'].median(),
    "Mode": df['sentiment_label'].mode()[0],
    "Standard Deviation": df['sentiment_label'].std(),
    "Standard Error": df['sentiment_label'].sem(),
    "Skewness": df['sentiment_label'].skew(),
    "Kurtosis": df['sentiment_label'].kurt(),
    "25th Percentile": df['sentiment_label'].quantile(0.25),
    "50th Percentile (Median)": df['sentiment_label'].quantile(0.50),
    "75th Percentile": df['sentiment_label'].quantile(0.75),
}

# Convert to DataFrame
descriptive_stats_df = pd.DataFrame(descriptive_stats, index=[0])

# Save to CSV
output_file = "sentiment_descriptive_statistics.csv"
descriptive_stats_df.to_csv(output_file, index=False)

print(f"Descriptive statistics for 'sentiment_label' saved to: {output_file}")


Descriptive statistics for 'sentiment_label' saved to: sentiment_descriptive_statistics.csv
